In [20]:
import os
import psycopg2
from psycopg2 import Error
from dotenv import load_dotenv
from contextlib import contextmanager
import pandas as pd
import numpy as np
from sqlalchemy import create_engine



In [23]:
db_params = {
    'host': os.getenv('DB_HOST'),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT', '5432')
}

In [16]:
def test_db_connection():
    """Test database connection and return connection status"""
    load_dotenv()
    try:
        # Connection parameters
        conn_params = {
            'host': os.getenv('DB_HOST', 'localhost'),
            'database': os.getenv('DB_NAME'),
            'user': os.getenv('DB_USER'),
            'password': os.getenv('DB_PASSWORD'),
            'port': os.getenv('DB_PORT', '5432')
        }
        
        # Try to establish a connection
        connection = psycopg2.connect(**conn_params)
        
        # Create a cursor and execute a simple query
        cursor = connection.cursor()
        cursor.execute('SELECT version();')
        db_version = cursor.fetchone()
        
        print("Successfully connected to the database!")
        print(f"PostgreSQL version: {db_version[0]}")
        
        cursor.close()
        return True, connection
        
    except Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        return False, None
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return False, None

In [21]:
connection = test_db_connection()

Successfully connected to the database!
PostgreSQL version: PostgreSQL 15.8 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit


In [24]:
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")


In [ ]:
query = """
SELECT 
    table_schema,
    table_name,
    (SELECT count(*) FROM information_schema.columns c 
     WHERE c.table_schema=t.table_schema AND c.table_name=t.table_name) as column_count
FROM information_schema.tables t
WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_schema, table_name;
"""

df_tables = pd.read_sql(query, engine)
df_tables

In [33]:
# For ciudades
df_ciudades = pd.read_sql("SELECT * FROM calculator.ciudades", engine)

# For dac
df_dac = pd.read_sql("SELECT * FROM calculator.dac", engine)

# For estaciones
df_estaciones = pd.read_sql("SELECT * FROM calculator.estaciones", engine)

# For geodata
df_geodata = pd.read_sql("SELECT * FROM calculator.geodata", engine)

# For log
df_log = pd.read_sql("SELECT * FROM calculator.log", engine)

# For municipios
df_municipios = pd.read_sql("SELECT * FROM calculator.municipios", engine)

# For t1
df_t1 = pd.read_sql("SELECT * FROM calculator.t1", engine)

# For tsb
df_tsb = pd.read_sql("SELECT * FROM calculator.tsb", engine)

In [34]:
df_dac.head()

,Month,Year,Value,Unit,Region,Segment,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,enero,2002,$32.1800,$/Cliente,Baja California,Fijo,69f7e611-27b6-457e-baf6-6101b15754e2,2024-09-05 17:09:23+00:00,{'changes': []}
1,enero,2002,$1.6280,$/kWh,Baja California,Energía,08950982-cfc6-4b65-920e-9fd47ce9066a,2024-09-05 17:09:23+00:00,{'changes': []}
2,enero,2002,$1.7910,$/kWh,Baja California,Verano,138045d4-e8e1-4b23-bae8-92b9148f4564,2024-09-05 17:09:23+00:00,{'changes': []}
3,enero,2002,$32.1800,$/Cliente,Baja California Sur,Fijo,f316855b-8e7d-4a94-a74c-5d67d17804d8,2024-09-05 17:09:23+00:00,{'changes': []}
4,enero,2002,$1.6280,$/kWh,Baja California Sur,Energía,b2b06c57-a401-4eea-bf8f-f51418bb9534,2024-09-05 17:09:23+00:00,{'changes': []}


In [32]:
# get the last row of the dac table
dac_last_row = df_dac.tail(1)
print(dac_last_row)

             Mes   Ano    Valor Unidad      Region Segmento  \
4958  septiembre  2024  $5.4400  $/kWh  Peninsular  Energia   

                           _airbyte_raw_id     _airbyte_extracted_at  \
4958  42addb25-9d47-4f1e-9a1c-f19e3c41e522 2024-09-05 17:09:24+00:00   

        _airbyte_meta  
4958  {'changes': []}  
